In [1]:
import time
import datetime
import requests
import csv

In [2]:
#!pip install tqdm
from tqdm import notebook

In [3]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [4]:
api_request_limit_per_hour = 4999
error_url = []
limit_check_hour = datetime.datetime.now().hour
can_continue = True

def HasItPassLimitHour(current_hour):
    return limit_check_hour != current_hour

def CheckRequestLimit():
    global api_request_limit_per_hour
    if api_request_limit_per_hour > 0:
        can_continue = True
    else:
        current_hour = datetime.datetime.now().hour
        if HasItPassLimitHour(current_hour) == True:
            limit_check_hour = datetime.datetime.now().hour
            api_request_limit_per_hour = 4999
            can_continue = True
        else:
            can_continue = False
    
def FetchDataFromGitHubRestApi(url):
    global api_request_limit_per_hour
    CheckRequestLimit()
    while can_continue == False:
        time.sleep(60)
        CheckRequestLimit()
    response = session.get(url)
    api_request_limit_per_hour -= 1
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [5]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

In [6]:
all_username = [x.strip() for x in all_username]

In [7]:
not_a_user_or_no_data = []

with open('all_user_profile_data.csv', 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "type", "hireable", "location", "followers", "following", "public_repos", "public_gists", "created_at", "updated_at"])

    c = 0
    for username in notebook.tqdm(all_username):

        try:
            user_alias = "usr"+str(c)
            
            profile_data = FetchDataFromGitHubRestApi("https://api.github.com/users/"+username)
    
            type_ = Select("type", profile_data)
            location = Select("location", profile_data)
            hireable = Select("hireable", profile_data)
            public_repos = Select("public_repos", profile_data)
            public_gists = Select("public_gists", profile_data)
            followers = Select("followers", profile_data)
            following = Select("following", profile_data)
            created_at = Select("created_at", profile_data)
            updated_at = Select("updated_at", profile_data)

            if type_ == "User":
                writer.writerow([user_alias, type_, hireable, location, followers, following, public_repos, public_gists, created_at, updated_at])
                c += 1
            else:
                not_a_user_or_no_data.append(username)
       
        except:
            not_a_user_or_no_data.append(username)

print("done!")

  0%|          | 0/8932 [00:00<?, ?it/s]

done!


In [8]:
lines = [x+"\n" for x in not_a_user_or_no_data]
with open('not_a_user_or_no_data.txt', 'w', newline='', encoding='utf8') as f:
    for line in lines:
        f.write(line)
print("done!")

done!
